# Weather Trend Forecasting — Data Cleaning (Beginner Friendly)

**CSV size:** ~24 MB → normal loading is fine.

This notebook teaches you, step by step, how to clean the Kaggle dataset **Global Weather Repository**.

We strictly separate **explanations (Markdown cells like this one)** from **runnable Python code (Code cells)**.

### What you will do
1. Load the raw CSV
2. Inspect columns and a small sample
3. Normalize column names
4. Find and parse the timestamp (using `lastupdated` or similar)
5. Handle missing values (numeric & categorical)
6. Clip extreme outliers
7. Aggregate to **daily level** (per city/country if available)
8. Save the cleaned CSV for EDA/modeling

**Inputs**: `data/Global_Weather_Repository.csv`  
**Outputs**: `data/cleaned_weather.csv`


## 0) (Optional) Install dependencies

If your environment is missing packages, remove the `#` and run the cell once.

- `pandas` handles tables
- `numpy` helps with numbers
- `matplotlib` draws charts


In [ ]:

# %pip install pandas numpy matplotlib pyarrow


## 1) Import libraries and define file paths

**Explanation of each line below:**
- `import pandas as pd` — Load the **pandas** library and give it a short name `pd`. We'll use `pd.read_csv(...)` and so on.
- `import numpy as np` — Load **numpy** as `np` for numeric helpers.
- `from pathlib import Path` — `Path` is a simple way to work with file system paths (like `data/myfile.csv`).
- `RAW = Path("...")` — Define where the raw CSV lives. We use a Path object instead of a plain string.
- `CLEAN = Path("...")` — Define where the cleaned CSV should be saved.
- `assert RAW.exists(), "..."` — If the file doesn’t exist, **stop** and show a helpful message.
- `print("Raw CSV path:", RAW.resolve())` — Show the absolute path, so you know what file we are reading.


In [ ]:

import pandas as pd      # Data handling
import numpy as np       # Numeric helpers
from pathlib import Path # File path utilities

RAW = Path("data/Global_Weather_Repository.csv")   # Location of your raw CSV
CLEAN = Path("data/cleaned_weather.csv")           # Where the cleaned file will be saved

# Safety check: stop early if the file is missing
assert RAW.exists(), f"Raw CSV not found: {RAW}. Please place the Kaggle file in the data/ folder."
print("Raw CSV path:", RAW.resolve())


## 2) Quick peek at the data (first 5 rows)

**Explanation of each line below:**
- `pd.read_csv(RAW, nrows=5, low_memory=False)` — Read only **5 rows** from the CSV to preview its structure.
- `display(sample.head())` — Nicely show those 5 rows.
- `print("Columns ...")` — Print a list of the first 20 column names so you can see what's inside.


In [ ]:

sample = pd.read_csv(RAW, nrows=5, low_memory=False)  # Load only 5 rows to preview
display(sample.head())                                 # Show the first 5 rows
print("Columns (first 20):", list(sample.columns)[:20], "...")  # Show some column names


## 3) Load the full dataset (24 MB is safe to load directly)

**Explanation of each line below:**
- `pd.read_csv(RAW, low_memory=False)` — Load the full CSV into a dataframe `df`.
- `print("Shape:", df.shape)` — Print how many rows and columns we have.


In [ ]:

df = pd.read_csv(RAW, low_memory=False)  # Load all rows
print("Shape (rows, cols):", df.shape)   # Show dataset dimensions


## 4) Normalize column names and parse the timestamp

Different versions of the dataset may use different column names (e.g., `LastUpdated`, `lastupdated`, `date`).  
We will:
1. Make all column names **lowercase** and replace spaces with underscores (easier to type).
2. Search a list of **candidate names** to find the timestamp column.
3. Convert that column to a real time type (`datetime`) in **UTC** and call it `timestamp`.
4. Drop rows where the time could not be parsed (rare but safe).
5. Sort the table by time.

**Explanation of each line below:**
- `df.columns = [...]` — Create a new list of cleaned column names using a **list comprehension**.
- `dt_candidates = [...]` — A list of possible time column names (we don’t know the exact spelling in your file).
- `dt_col = next((c for c in dt_candidates if c in df.columns), None)` — Pick the first candidate that exists in your data.
- `assert dt_col` — If none found, stop and instruct you to check the file.
- `pd.to_datetime(..., errors="coerce", utc=True)` — Convert text to a real datetime; bad values become `NaT` (missing time); use UTC timezone.
- `dropna(subset=["timestamp"])` — Remove rows with missing timestamps.
- `sort_values("timestamp")` — Order rows by time.


In [ ]:

# 1) Normalize column names: trim spaces, lowercase, replace spaces with underscores
df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]

# 2) Try multiple possible time column names
dt_candidates = ["lastupdated","last_updated","lastupdate","timestamp","date","datetime"]
dt_col = next((c for c in dt_candidates if c in df.columns), None)
assert dt_col, f"No datetime column found. Tried: {dt_candidates}"

# 3) Parse to pandas datetime in UTC; invalid parses become NaT (missing)
df["timestamp"] = pd.to_datetime(df[dt_col], errors="coerce", utc=True)

# 4) Drop rows with invalid/missing timestamps
before = len(df)
df = df.dropna(subset=["timestamp"]).sort_values("timestamp")
after = len(df)
print(f"Dropped {before - after} rows with invalid timestamps.")
df[["timestamp"]].head()


## 5) Handle missing values (NaNs)

We will fill missing values so later steps don’t break.

**Strategy:**
- For **numeric columns**: fill with the **median** (robust to outliers).
- For **categorical/text columns**: fill with the **mode** (most frequent value).

**Explanation of each line below:**
- `select_dtypes(include=[np.number])` — Find numeric columns.
- `cat_cols = [c for c ...]` — Everything else is treated as categorical.
- Loop over the columns and fill missing values appropriately.


In [ ]:

# Identify numeric and categorical columns
num_cols = df.select_dtypes(include=[np.number]).columns            # numeric columns like temperature, precipitation, etc.
cat_cols = [c for c in df.columns if c not in num_cols]             # non-numeric columns like city, country

# Fill numeric NaNs with the median of that column
for c in num_cols:
    if df[c].isna().any():
        df[c] = df[c].fillna(df[c].median())

# Fill categorical NaNs with the most frequent value (mode) of that column
for c in cat_cols:
    if df[c].isna().any():
        mode = df[c].mode()
        if not mode.empty:
            df[c] = df[c].fillna(mode.iloc[0])


## 6) Reduce the impact of extreme outliers

We **clip** extreme values to a safe range (between the 1st and 99th percentile).  
This keeps unusual spikes from dominating your analysis, while still keeping the row.

**Explanation of each line below:**
- `quantile([0.01, 0.99])` — Calculate the 1% and 99% cutoff points for a column.
- `clip(lower=..., upper=...)` — Replace values below/above the cutoffs with the cutoff values.


In [ ]:

for c in num_cols:
    q1, q99 = df[c].quantile([0.01, 0.99])  # 1st and 99th percentiles
    df[c] = df[c].clip(lower=q1, upper=q99) # Clip extreme values


## 7) Aggregate to **daily** level (optionally per city/country)

Your dataset may have multiple records inside the same day.  
We will:
1. Create a `date` column from `timestamp`.
2. Define **grouping keys**: always `date`, plus `city`/`country` if those columns exist.
3. Group and compute the **mean** for numeric columns.

**Explanation of each line below:**
- `df["timestamp"].dt.date` — Extract the date (year-month-day) from the full timestamp.
- `group_keys = [...]` — Start with `date`; add `city`/`country` if available.
- `groupby(...).mean(numeric_only=True)` — Average numeric values inside the same group.
- `reset_index()` — Turn the group keys back into normal columns.
- Re-create `timestamp` from `date` so later plotting is easy.


In [ ]:

# 1) Create a pure date column
df["date"] = df["timestamp"].dt.date

# 2) Build grouping keys
group_keys = ["date"]
if "city" in df.columns: group_keys.append("city")
if "country" in df.columns: group_keys.append("country")

# 3) Group and average numeric values
df_daily = df.groupby(group_keys).mean(numeric_only=True).reset_index()

# Recreate a timestamp column (midnight of that day)
import pandas as pd
df_daily["timestamp"] = pd.to_datetime(df_daily["date"])  # converts 'date' back to a datetime
df_daily = df_daily.drop(columns=["date"])

print("Daily shape:", df_daily.shape)
df_daily.head()


## 8) Save the cleaned dataset

We save the result to `data/cleaned_weather.csv`.  
This file will be used by EDA and forecasting notebooks.


In [ ]:

# Ensure the output folder exists; then write CSV
CLEAN.parent.mkdir(parents=True, exist_ok=True)
df_daily.to_csv(CLEAN, index=False)
print("Saved cleaned dataset to:", CLEAN.resolve())


## 9) (Optional) Quick visual check (matplotlib only)

We try to plot **temperature** and **precipitation** trends for one city, if such columns exist.

- We **do not** use seaborn here (to keep it simple and dependency-light).
- Each chart is drawn in its **own figure**.


In [ ]:

import matplotlib.pyplot as plt

# Candidate column names that commonly appear in this dataset
temp_candidates = ["temp_c","temperature_c","temperature","avg_temp_c","temp"]
precip_candidates = ["precip_mm","precipitation","rain_mm","rain"]

# Detect which columns exist in your cleaned daily data
temp_col = next((c for c in temp_candidates if c in df_daily.columns), None)
precip_col = next((c for c in precip_candidates if c in df_daily.columns), None)

# If we have a city and temperature column, pick the most frequent city and plot its trends
if "city" in df_daily.columns and temp_col:
    city = df_daily["city"].astype(str).value_counts().index[0]
    sub = df_daily[df_daily["city"]==city].sort_values("timestamp")
    
    # Temperature trend
    plt.figure()
    plt.plot(sub["timestamp"], sub[temp_col])
    plt.title(f"Temperature trend — {city}")
    plt.xlabel("Date")
    plt.ylabel(temp_col)
    plt.show()
    
    # Precipitation trend (if available)
    if precip_col:
        plt.figure()
        plt.plot(sub["timestamp"], sub[precip_col])
        plt.title(f"Precipitation trend — {city}")
        plt.xlabel("Date")
        plt.ylabel(precip_col)
        plt.show()
else:
    print("Skipping quick plots: missing 'city' or temperature column.")
